テーブル作成とデータロードQiita [Db2 v12.1.2でベクトルデータが入るようになったので、SQLでいろいろやってみた](https://qiita.com/nishikyon/items/8fdb36aaffd59e617418)の「2. ALTER TABLEして、カラムのデータをベクトル化して追加」のコードです


# 03. ALTER TABLEして、カラムのデータをベクトル化して追加

### 1. Embeddingモデルを作成
- 現状はSQLだけではベクトルデータが作成できません。
- HuggingFaceのEmbeddingsモデル　ここでは`intfloat/multilingual-e5-large`を使います

- https://huggingface.co/intfloat/multilingual-e5-large


In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from tqdm import tqdm

embeddings = HuggingFaceEmbeddings(model_name="intfloat/multilingual-e5-large")

### 2. Jupyter NotebookでDb2 Magic Commands拡張機能を有効にする
Jupyter NotebookからSQLをわかりやすく打てるように、Db2 Magic Commands拡張機能を有効にします。(Db2 ベクトルデータとは関係ないです。）

参考: [Jupyter NotebookからDb2に簡単アクセス](https://qiita.com/nishikyon/items/b0d0262950e890349f29)

In [ ]:
import os
# Jupyter NotebookでDb2 Magic Commands拡張機能を有効にする
if not os.path.isfile('db2.ipynb'):
    os.system('wget https://raw.githubusercontent.com/IBM/db2-jupyter/master/db2.ipynb')

%run db2.ipynb

### 3. Db2へ接続
.envファイル(または環境変数)から必要情報を読み取って、Db2に接続します。

In [ ]:
from dotenv import dotenv_values

# .envファイルを読み込む
# 事前にsample_env を 参考に .envを作成しておく

con_str = dotenv_values(".env")
database = con_str.get("database")
user=con_str.get("uid")
password=con_str.get("pwd")
host=con_str.get("hostname")
port=con_str.get("port")

# SSL接続の場合はSSL TRUEを最後に付与する
%sql CONNECT TO {database} USER {user} USING {password} HOST {host} port {port}

### 4. SELECTで確認:  VECTEST.EXPO_INFO

SELECT文でVECTEST.EXPO_INFOの中身を確認してみます

In [ ]:
%sql SELECT * FROM VECTEST.EXPO_INFO

### 5.  VECTEST.EXPO_INFO に　`VECTOR` 列の追加

ベクトルデータを入れる列はデータ型[VECTOR](https://www.ibm.com/docs/ja/db2/12.1.0?topic=list-vector-values#c_vectors__title__4)で定義します。

Syntax
```
VECTOR (< ベクトルデータの次元数 > ,< coordinate-type　FLOAT32 または INT8 >)
```
SQLのALTER TABLEコマンドで追加します(DBeaverなどで直接SQLを実行してもOK):

In [ ]:
%%sql
ALTER TABLE VECTEST.EXPO_INFO
ADD COLUMN EMBEDDING VECTOR(1024, FLOAT32);

確認(DBeaverなどで直接SQLを実行してもOK):

In [ ]:
# 確認
%sql SELECT * FROM VECTEST.EXPO_INFO FETCH FIRST 3 ROWS ONLY

テーブル構成の確認 (これはDBeaverでは動作しません、Db2 CLPであればOK):

In [ ]:
%sql DESCRIBE TABLE VECTEST.EXPO_INFO

### 6.  VECTEST.EXPO_INFO から　特徴を表す列をベクトル化する

ここでは以下の列をJSON化して、ベクトル化します:

- 見どころ
- 詳細
- おとな向け評価
- おとなコメント
- 中高生向け評価
- 中高生コメント
- 子連れ向け評価
- 子連れコメント

#### 6-1. 特徴を表す列をJSONに加工する
[JSON_OBJECT関数](https://www.ibm.com/docs/ja/db2/12.1.0?topic=sf-json-object)を使ってSQLでJSONにします。

[JSON_OBJECT関数](https://www.ibm.com/docs/ja/db2/12.1.0?topic=sf-json-object)がDb2 Magic Commandでは使えなかったため、sqlalchemyでSQLを実行しています。

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
from urllib.parse import quote_plus

# DB2の接続情報を設定
password_q = quote_plus(password)

# SQLAlchemyのエンジンを作成
url = f"ibm_db_sa://{user}:{password_q}@{host}:{port}/{database}"
engine = create_engine(url)

sql_str = """
SELECT 
       ID,
       JSON_OBJECT(
         '見どころ' VALUE 見どころ,
         '詳細' VALUE 詳細,
         'おとな向け評価' VALUE おとな向け評価,
         'おとなコメント' VALUE おとなコメント,
         '中高生向け評価' VALUE 中高生向け評価,
         '中高生コメント' VALUE 中高生コメント,
         '子連れ向け評価' VALUE 子連れ向け評価,
         '子連れコメント' VALUE 子連れコメント
       ) AS JSON_RESULT
FROM VECTEST.EXPO_INFO;
"""
# pandas.read_sql() を使用してpandas Dataframe型のdf_vecにSELECTの結果を入れる
df_vec_text = pd.read_sql(sql_str, engine)

# 最初の1行のJSON表示
print( df_vec_text.at[0,'json_result'])

# DataFrame表示
df_vec_text

#### 6-2. df_vec_textのjson_result列をベクトル化

id(主キー)とson_result列をベクトル化したデータEMBEDDING列の2列のDataFrameを作ります。

In [ ]:
# json_result列をベクトル化してEMBEDDING列を作り、
# id(主キー)とson_result列をベクトル化したデータEMBEDDING列の2列のDataFrameを作ります。
list_vec_text = df_vec_text['json_result'].tolist()
expo_vectors = embeddings.embed_documents(texts=list_vec_text)
df_vec_text['EMBEDDING'] = expo_vectors
df_vec_text['EMBEDDING'] =df_vec_text['EMBEDDING'].apply(lambda x: '[' + ', '.join(map(str, x)) + ']')
df_vec_text.drop(columns=['json_result'], inplace=True)

df_vec_text

### 6-3. ベクトル化した値でEMBEDDING列を更新

更新はibm_dbライブラリを使用し、SQLのMERGE文で実施します。

DataFrameの値を使用した更新は、MERGE文が速いとのことです。

In [ ]:
# そして今度は更新用にibm_dbで接続を取得します ....
    
import pandas as pd
import ibm_db

# Db2への接続（接続情報は適宜変更してください）
conn_str = f"DATABASE={database};HOSTNAME={host};PORT={port};PROTOCOL=TCPIP;UID={user};PWD={password};"
DB_connection_I = ibm_db.connect(conn_str, "", "")


In [ ]:

# MERGE文のVALUES部分を構築（行数に応じて ? を繰り返す）
values_clause = ",".join(["(?, ?)"] * len(df_vec_text))
# print(f"values_clause: {values_clause}")


# MERGE文の定義
sql = f"""
MERGE INTO  VECTEST.EXPO_INFO AS T1
USING (VALUES {values_clause}) AS T2(ID, EMBEDDING)
ON T1.ID = T2.ID
WHEN MATCHED THEN
  UPDATE SET T1.EMBEDDING = VECTOR(T2.EMBEDDING,  1024, FLOAT32)
"""

# プレースホルダーに渡す値をフラットなリストに変換
params = [item for row in df_vec_text.itertuples(index=False) for item in row]

# ステートメント準備と実行
stmt = ibm_db.prepare(DB_connection_I, sql)
ibm_db.execute(stmt, tuple(params))


確認　(DBeaverなどで直接SQLを実行してもOK):

In [ ]:
%sql SELECT * FROM VECTEST.EXPO_INFO FETCH FIRST 3 ROWS ONLY